In [1]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session
import pandas as pd
import sqlite3
import csv

In [2]:
Base = declarative_base()

In [3]:
class Wines(Base):
    __tablename__ = 'wines'
    index = Column(Integer, primary_key = True, default = 0)
    country = Column(String(255))
    description = Column(String(255))
    points = Column(Integer)
    price = Column(Integer)
    province = Column(String(255))
    region_1 = Column(String(255))
    region_2 = Column(String(255))
    title = Column(String(255))
    variety = Column(String(255))
    winery = Column(String(255))

In [44]:
class StopWords(Base):
    __tablename__ = 'stopWords'
    index = Column(Integer, primary_key = True, default = 0)
    stopWords = Column(String(255))


In [45]:
dialect = 'sqlite'
user = 'root'
password = 'wineplease'
host = 'localhost'
port = 3306
database = 'wines.sqlite'

In [46]:
engine = create_engine(f'{dialect}:///{database}')

In [47]:
Base.metadata.create_all(engine)

In [18]:
wine_df = pd.read_csv("winemag-data-130k-v3.csv")
varieties = wine_df['variety'].value_counts()

In [19]:
top_wines_df = wine_df.loc[wine_df['variety'].isin(varieties.axes[0][:20])]

In [23]:
wine_stop_words = []
for variety in top_wines_df['variety'].unique():
    for word in variety.split(' '):
        wine_stop_words.append(word.lower())
wine_stop_words = pd.Series(data=wine_stop_words).unique()

In [24]:
from sklearn.feature_extraction import text

In [25]:
stop_word_list = text.ENGLISH_STOP_WORDS.union(wine_stop_words)

In [34]:
stop_word_df = pd.DataFrame(columns=['stopWords'])

In [40]:
stop_word_df['stopWords'] = list(stop_word_list)

In [16]:
session = Session(bind=engine)
conn = sqlite3.connect('wines.sqlite')
cur = conn.cursor()

In [20]:
top_wines_df.to_sql('wines', con=engine, index=False,if_exists='append')

In [48]:
stop_word_df.to_sql('stopWords', con=engine, index=False, if_exists='append')

In [51]:
session.commit()

In [21]:
pd.read_sql('wines', con=engine)

,index,country,description,points,price,province,region_1,region_2,title,variety,winery
0,1,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,None,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,2,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,None,None,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,3,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,4,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,None,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,5,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,6,France,This has great depth of flavor with its fresh ...,87,27.0,Alsace,Alsace,None,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
6,7,US,"Soft, supple plum envelopes an oaky structure ...",87,19.0,California,Napa Valley,Napa,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
7,8,US,"Slightly reduced, this wine offers a chalky, t...",87,34.0,California,Alexander Valley,Sonoma,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
8,9,US,Building on 150 years and six generations of w...,87,12.0,California,Central Coast,Central Coast,Mirassou 2012 Chardonnay (Central Coast),Chardonnay,Mirassou
9,10,Germany,Zesty orange peels and apple notes abound in t...,87,24.0,Mosel,None,None,Richard Böcking 2013 Devon Riesling (Mosel),Riesling,Richard Böcking


In [50]:
pd.read_sql('stopWords', con=engine)

,index,stopWords
0,1,whoever
1,2,through
2,3,that
3,4,amount
4,5,whose
5,6,also
6,7,between
7,8,sixty
8,9,call
9,10,front


In [52]:
stop_words = pd.read_sql('stopWords', con=engine)

In [54]:
stop_words['stopWords']

0         whoever
1         through
2            that
3          amount
4           whose
5            also
6         between
7           sixty
8            call
9           front
10     afterwards
11             me
12        because
13           will
14          these
15          their
16           were
17       amoungst
18           from
19     everywhere
20      something
21            any
22            and
23          forty
24           none
25     whereafter
26         others
27          under
28           both
29          after
          ...    
311            it
312      sometime
313          gris
314        though
315          mine
316          ours
317    sangiovese
318         among
319           nor
320        thence
321     otherwise
322          have
323         empty
324           who
325        eleven
326        hereby
327        myself
328          find
329     elsewhere
330           her
331     somewhere
332      together
333            on
334           inc
335       